<img src="https://oasislmf.org/packages/oasis_theme_package/themes/oasis_theme/assets/src/oasis-lmf-colour.png" alt="Oasis LMF logo" width="250" align="left"/>
<br><br><br>

# Excercise 2:   Introduction to Oasis model files and formats.

The Oasis kernel provides a robust loss simulation engine for catastrophe modelling. Insurance practitioners are used to dealing with losses arising from events. These losses are numbers, not distributions. Policy terms are applied to the losses individually and then aggregated and further conditions or reinsurances applied. Oasis takes the same perspective, which is to generate individual losses from the probability distributions. The way to achieve this is random sampling called “Monte-Carlo” sampling from the use of random numbers, as if from a roulette wheel, to solve equations that are otherwise intractable.

Modelled and empirical intensities and damage responses can show significant uncertainty, Sometimes this uncertainty is multi-modal, meaning that there can be different peaks of behaviour rather than just a single central behaviour. Moreover, the definition of the source insured interest characteristics, such as location or occupancy or construction, can be imprecise. The associated values for event intensities and consequential damages can therefore be varied and their uncertainty can be represented in general as probability distributions rather than point values. The design of Oasis therefore makes no assumptions about the probability distributions and instead treats all probability distributions as probability masses in discrete bins. This includes closed interval point bins such as the values [0,0] for no damage and [1,1] for total damage.

The simulation approach taken by the Oasis calculation kernel computes a single cumulative distribution function (CDF) for the damage by “convolving” the binned intensity distribution with the vulnerability matrices. Sampling can then be done against the CDF. 

<img src="images/simulation_methodology.png" alt="Oasis simulation methodology" width="600"/>

The Oasis kernel requires a standard set of files for capturing the hazard footprints and vulnerability data.

<img src="images/oasis_model_files.png" alt="Oasis model files" width="600"/>

The files are:

#### area peril dictionary
    The meta-data that describes the model specific geo-spatial grid. This can be a set of points, a regular grid or a variable resolutiuon grid.
#### hazard
    The hazard values for each impacted area peril cell for each event in the stochastic catalogue.
#### event
    The list of events in the stochastic catalogue. Event files can be use to distinguish event types, such as historical.
#### intensity bin dictionary
    The meta-data that descibes the hazard intensities corresponding to the bins.
#### vulnerability
    The vulnerability data. 
#### vulnerability dictionary
    The meta-data that descibes the vulnerability data, in particular mapping particular curves to particular exposure attributes.
#### damage bin dictionary
    The meta-data tha descibes the damage percentages corresponding to the bins.


## Excercise goals
* Understand the Oasis model files
* Use Python code to visualise the model files for an example model
* Use Python code to visualise run and view the results of an analysis


In [1]:
%config IPCompleter.greedy=True

In [2]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import geopandas as gpd
import math
import numpy as np
import json
import seaborn as sns
import folium
from folium.plugins import HeatMap
import os
from shapely.geometry import Point, Polygon
from descartes import PolygonPatch
from mpl_toolkits.axes_grid1 import AxesGrid
from matplotlib.offsetbox import AnchoredText
import branca.colormap as cm

In [3]:
area_peril_dictionary = pd.read_csv("./keys_data/MEEQ/area_peril_dict.csv")
area_peril_dictionary.head()

,area_peril_id,longitude,latitude
0,1005275201,35.4500,33.7167
1,1005274201,35.5167,33.7167
2,1005258001,35.4542,33.7292
3,1005257901,35.4542,33.7375
4,1005254601,35.5208,33.7375


In [50]:
m = folium.Map(location=[	33.85, 35.50], zoom_start=12, tiles='cartodbpositron')
for i, row in area_peril_dictionary.iterrows():
    folium.CircleMarker(
        location=[row.latitude, row.longitude], radius=1).add_to(m)
m.fit_bounds(m.get_bounds())
m

In [26]:
intensity_bin_dictionary = pd.read_csv("./model_data/MEEQ/intensity_bin_dict.csv")
intensity_bin_dictionary.head()

,bin_id,bin_from,bin_to,interpolation,interval_type
0,1,-3.400,-2.765,-2.765,1201
1,2,-2.765,-2.676,-2.720,1201
2,3,-2.676,-2.586,-2.631,1201
3,4,-2.586,-2.495,-2.540,1201
4,5,-2.495,-2.403,-2.449,1201


Lets have a look at the data contained in the footprint file. 

In [23]:
footprints = pd.read_csv("./model_data/MEEQ/footprint_subset.csv")
footprints.head()

,event_id,area_peril_id,intensity_bin_id,probability
0,3342,1005237101,1,0.968867
1,3342,1005237101,2,0.008444
2,3342,1005237101,3,0.006488
3,3342,1005237101,4,0.004874
4,3342,1005237101,5,0.003579


In [24]:
area_peril_dictionary = pd.read_csv("./keys_data/MEEQ/area_peril_dict.csv")
area_peril_dictionary.head()

,area_peril_id,longitude,latitude
0,1005275201,35.4500,33.7167
1,1005274201,35.5167,33.7167
2,1005258001,35.4542,33.7292
3,1005257901,35.4542,33.7375
4,1005254601,35.5208,33.7375


In [53]:
event_id = 3342

footprints_with_hazard = footprints.merge(
    area_peril_dictionary, how='inner', 
    on='area_peril_id').merge(
    intensity_bin_dictionary, how="inner",
    left_on="intensity_bin_id", right_on="bin_id")[footprints_with_hazard.event_id == event_id]

linear = cm.LinearColormap(
    ['green', 'yellow', 'red'],
    vmin=min(footprints_with_hazard.interpolation), 
    vmax=max(footprints_with_hazard.interpolation))
m = folium.Map(location=[33.85, 35.50], zoom_start=12, tiles='cartodbpositron')
for i, row in footprints_with_hazard.iterrows():
    c = linear(row.interpolation)
    folium.CircleMarker(
        location=[row.latitude, row.longitude], fill_color=c, radius=5,
        weight=0, fill=True, fill_opacity=1.0).add_to(m)
linear.caption = 'log PGA'
m.fit_bounds(m.get_bounds())
m.add_child(linear)
m

In [28]:
damage_bin_dictionary = pd.read_csv("./model_data/MEEQ/damage_bin_dict.csv")
damage_bin_dictionary.head()

,bin_id,bin_from,bin_to,interpolation,interval_type
0,1,0.000,0.000,0.0000,1201
1,2,0.000,0.001,0.0005,1201
2,3,0.001,0.020,0.0105,1201
3,4,0.020,0.040,0.0300,1201
4,5,0.040,0.060,0.0500,1201


In [29]:
vulnerabilities = pd.read_csv("./model_data/MEEQ/vulnerability_subset.csv")
vulnerabilities.head()

,vulnerability_id,intensity_bin_id,damage_bin_id,probability
0,712,1,1,1.000000
1,712,2,1,1.000000
2,712,3,2,0.516807
3,712,3,3,0.464853
4,712,3,4,0.013151


In [44]:
from numpy import linspace
from bokeh.io import output_file, show, output_notebook
from bokeh.models import ColumnDataSource, FixedTicker, PrintfTickFormatter
from bokeh.plotting import figure
from bokeh.sampledata.perceptions import probly

import colorcet as cc

def joy(category, data, scale=1):
    return list(zip([category]*len(data), scale*data))

num_intensity_bins = len(intensity_bin_dictionary.bin_id)
num_damage_bins = len(damage_bin_dictionary.bin_id)

cats = list(reversed(probly.keys()))
x = linspace(1, num_damage_bins+1, num_damage_bins+1)
source = ColumnDataSource(data=dict(x=x))
intensity_range = list([str(i) for i in intensity_bin_dictionary.interpolation])
intensity_range = list([str(i) for i in intensity_bin_dictionary.bin_id])
p = figure(y_range=intensity_range, plot_width=900, x_range=(0, num_damage_bins), toolbar_location=None)

data = vulnerabilities[vulnerabilities.vulnerability_id==712]
for i, intensity_bin_id in enumerate(reversed(intensity_range)):        
    all_damage_bins=pd.DataFrame({"damage_bin_id": list(range(0,num_damage_bins+1))})
    d = all_damage_bins.merge(data[data.intensity_bin_id==int(intensity_bin_id)], on="damage_bin_id", how="outer")
    d = d.fillna(0)
    if len(data[data.intensity_bin_id==int(intensity_bin_id)]) == 1:
        d.loc[0, 'probability'] = 0.001
        d.loc[2, 'probability'] = 0.001
    y = joy(intensity_bin_id, d.probability, 15)
    source.add(y, intensity_bin_id)
    p.patch('x', intensity_bin_id, alpha=0.6, line_color="black", source=source)

p.outline_line_color = None
p.background_fill_color = "#efefef"

#p.xaxis.ticker = FixedTicker(ticks=list(range(0, 101, 10)))
#p.xaxis.formatter = PrintfTickFormatter(format="%d%%")
#p.xaxis.formatter = PrintfTickFormatter(format="%i%")

p.ygrid.grid_line_color = None
p.xgrid.grid_line_color = "#dddddd"
p.xgrid.ticker = p.xaxis[0].ticker

p.axis.minor_tick_line_color = None
p.axis.major_tick_line_color = None
p.axis.axis_line_color = None

p.y_range.range_padding = 0.12
output_notebook()
show(p)

Loading BokehJS ...

To run the model we need some test exxposure data. Lets have a look at an example Location and Account file. 

In [45]:
test_locations = pd.read_csv('./source_data/MEEQ_loc.csv')
test_locations.head()

,ACCNTNUM,LOCNUM,LATITUDE,LONGITUDE,POSTCODE,ADDRMATCH,STATE,STATECODE,COUNTY,COUNTYCODE,...,EQSITEDED,EQSITEDCUR,EQCOMBINEDLIM,EQCOMBINEDLCUR,EQCOMBINEDDED,EQCOMBINEDDCUR,COND1TYPE,COND1NAME,COND1LIMIT,COND1DEDUCTIBLE
0,AEEQ_AbuD_flat_com__noYB_f_buil,1,24.4539,54.3773,XXX,0,0,0,0,0,...,0,USD,0,USD,0,USD,XXX,XXX,0,0
1,AEEQ_AbuD_flat_res__noYB_f_buil,1,24.4539,54.3773,XXX,0,0,0,0,0,...,0,USD,0,USD,0,USD,XXX,XXX,0,0
2,AEEQ_FlatCB_ForMZ,2,24.4539,54.3773,XXX,0,0,0,0,0,...,0,USD,0,USD,0,USD,XXX,XXX,0,0
3,AEEQ_FlatRB_ForMZ,2,24.4539,54.3773,XXX,0,0,0,0,0,...,0,USD,0,USD,0,USD,XXX,XXX,0,0
4,ForMZ__AEEQ_AbuCB,1,24.4539,54.3773,XXX,0,0,0,0,0,...,0,USD,0,USD,0,USD,XXX,XXX,0,0


To run the model, we also need to define some analysis settings. Lets have a look at an example settings file.

In [46]:
with open('./analysis_settings.json', 'r') as myfile:
    analysis_settings=json.loads(myfile.read().replace('\n', ''))
print(json.dumps(analysis_settings, indent=True))

{
 "analysis_settings": {
  "analysis_tag": 1,
  "gul_output": true,
  "gul_summaries": [
   {
    "aalcalc": true,
    "eltcalc": true,
    "id": 1,
    "lec_output": true,
    "leccalc": {
     "outputs": {
      "full_uncertainty_aep": true,
      "full_uncertainty_oep": true
     },
     "return_period_file": true
    }
   }
  ],
  "gul_threshold": 0,
  "il_summaries": [],
  "model_settings": {
   "event_set": "",
   "event_occurrence_id": ""
  },
  "model_version_id": "MEEQ",
  "module_supplier_id": "Catrisks",
  "number_of_samples": 10,
  "source_tag": "MEEQ"
 }
}


In [47]:
! rm -rf /tmp/analysis_test
! oasislmf model run -C oasislmf.json -r /tmp/analysis_test


Creating temporary folder /tmp/analysis_test/tmp for Oasis files

Getting model info and lookup
STARTED: oasislmf.keys.lookup.__init__
STARTED: oasislmf.keys.lookup.__init__
COMPLETED: oasislmf.keys.lookup.__init__ in 0.0s
STARTED: oasislmf.keys.lookup.__init__
STARTED: oasislmf.keys.lookup.__init__
COMPLETED: oasislmf.keys.lookup.__init__ in 0.0s
COMPLETED: oasislmf.keys.lookup.__init__ in 0.0s
STARTED: oasislmf.keys.lookup.__init__
STARTED: oasislmf.keys.lookup.__init__
COMPLETED: oasislmf.keys.lookup.__init__ in 0.0s
STARTED: oasislmf.keys.lookup.get_vulnerabilities
COMPLETED: oasislmf.keys.lookup.get_vulnerabilities in 0.02s
COMPLETED: oasislmf.keys.lookup.__init__ in 0.02s
COMPLETED: oasislmf.keys.lookup.__init__ in 0.02s
	{'supplier_id': 'Catrisks', 'model_id': 'MEEQ', 'model_version': '0.0.0.1'}, <oasislmf.keys.lookup.OasisLookup object at 0x7f6b0aac4a58>

Creating Oasis model object
	<class 'oasislmf.models.model.OasisModel'>: {'_supplier_id': 'Catrisks', '_model_id': 'MEEQ', 

STARTED: oasislmf.model_execution.runner.run
COMPLETED: oasislmf.model_execution.runner.run in 2.12s

Loss outputs generated in /tmp/analysis_test/output


In [49]:
from bokeh.io import show, output_file
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.plotting import figure
from bokeh.transform import factor_cmap

analysis_directory = "/tmp/analysis_test"
gul_aep = pd.read_csv(os.path.join(analysis_directory, "output", "gul_S1_leccalc_full_uncertainty_aep.csv"))
gul_oep = pd.read_csv(os.path.join(analysis_directory, "output", "gul_S1_leccalc_full_uncertainty_oep.csv"))
eps = pd.merge(gul_oep, gul_aep, on=["summary_id", "return_period"], suffixes=["_oep", "_aep"])
eps = eps.sort_values(by="return_period", ascending=True)
return_periods = eps.return_period
lec_types = ['OEP', 'AEP']
data = {'Return periods' : return_periods,
        'OEP': eps.loss_oep,
        'AEP': eps.loss_aep}
palette = ["#c9d9d3", "#718dbf"]
x = [ (str(return_period), lec_type) for return_period in return_periods for lec_type in lec_types ]
counts = sum(zip(data['OEP'], data['AEP']), ())
source = ColumnDataSource(data=dict(x=x, counts=counts))
p = figure(x_range=FactorRange(*x), plot_height=350, title="EP by return period",
           toolbar_location=None, tools="")
p.vbar(x='x', top='counts', width=0.9, source=source, line_color="white",
       fill_color=factor_cmap('x', palette=palette, factors=lec_types, start=1, end=2))
p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None
show(p)